# Content
1. [Inputs](#inputs)
    1. [Occurrence records](#occurrenceRecords)
    2. [Kernel density estimate and background points](#kdeBg)
    3. [Bioclimatic variables](#biovar)
2. [Maxent modelling](#maxent) <br>
3. [Maxent results](#results)

**All codes in this section were implemented in R unless otherwise stated**

## 1. Inputs <a id='inputs'></a>

### A. Occurrence records <a id='occurrenceRecords'></a>

Obtain species occurrence records

1. eBird basic data: https://ebird.org/data/download
2. Global Biodiversity Information Facility (GBIF): https://www.gbif.org/occurrence/search?occurrence_status=present&q=

Filter and thin occurrence records

In [ ]:
## load packages
library(tidyr)
library(spThin)
library(stringr)
library(sp)
library(rgdal)
library(spatialEco)
library(raster)

## import occurrence records
# eBird
data <- read.delim("ebd_species_relMonth-yyyy.txt", quote="") # import file 
data2 <- data[!duplicated(data[c("LATITUDE","LONGITUDE")]),] # remove duplicates based on latitude and longitude
# split date into year month day columns
ymd <- str_split_fixed(data2$OBSERVATION.DATE, "-", 3)
ymd2 <- as.data.frame(ymd)
colnames(ymd2) <- c("Year", "Month","Date")
ymd2$Year <- as.numeric(as.character(ymd2$Year))
ymd2$Month <- as.numeric(as.character(ymd2$Month))
ymd2$Date <- as.numeric(as.character(ymd2$Date))
data3 <- cbind(data2, ymd2)
data4 <- data3[data3$Month>3 & data3$Month<7, ] # subset records from Apr to Jun
data5 <- data4[data4$Year>1959 & data4$Year<1991, ] # subset records from 1960-1990
ebird <- data5[, c("LONGITUDE", "LATITUDE")] # create new dataframe with longitude and latitude

# GBIF
gdata <- read.csv(file = 'species-gbif.csv', header = TRUE) # import file
gdata2 <- gdata[!duplicated(gdata[c("decimalLatitude","decimalLongitude")]),] # remove duplicates based on latitude and longitude
# split date into year month day columns
library(stringr)
gdata2$year <- as.numeric(as.character(gdata2$year))
gdata2$month <- as.numeric(as.character(gdata2$month))
gdata2$day <- as.numeric(as.character(gdata2$day))
gdata3 <- gdata2[gdata2$month>3 & gdata2$month<7, ] # subset records from Apr to Jun
gdata4 <- gdata3[gdata3$year>1959 & gdata3$year<1991, ] # subset records from 1960-1990
gbif <- gdata4[, c("decimalLongitude", "decimalLatitude")] # create new dataframe with longitude and latitude
colnames(gbif) <- c("LONGITUDE", "LATITUDE")

## combine eBird and GBIF occurrence records
species <- rbind(ebird, gbif)
species2 <- species[!duplicated(species[c("LATITUDE","LONGITUDE")]),] # remove duplicates based on latitude and longitude
species3 <- species2 %>% drop_na(LONGITUDE) # remove NA rows

## remove points outside of IUCN/eBird breeding area
Breeding <- readOGR(dsn = "path/species", layer = "speciesBr")
species4 <- SpatialPoints(species3) # make into r SpatialPointsDataFrame
species5 <- erase.point(species4, Breeding, inside = FALSE)
species6 <- as.data.frame(species5)
species7 <- cbind(a = "Genus species", species6)
colnames(species7) <- c("species", "longitude", "latitude")

Downsample occurrence records for *N. phaeopus* and *N. arquata*

In [ ]:
# load packages
library(rgdal)
library(spatialEco)
library(sf)
library(maptools)
library(raster)
library(spatstat)

# import occurence file
occ <- read.csv(file = 'occurrence-species-all-1960to1990-AprJun.csv', header = TRUE)
occ1 <- occ[,-1]

# extract records inside and outside of Europe
occ1sp <- SpatialPoints(occ1)
eur <- readOGR(dsn = "shapefile/folder", layer = "Europe")
eurSp <- erase.point(occ1sp, eur, inside = FALSE)
othSp <- erase.point(occ1sp, eur, inside = TRUE)

# convert to dataframe
othOcc <- as.data.frame(othSp)
eurOcc <- as.data.frame(eurSp)

## calculate density of occurrence records
# outside Europe
othPPP <- as.ppp(othSp)
othDen <- intensity(othPPP)
# inside Europe
eurPPP <- as.ppp(eurSp)
eurDen <- intensity(eurPPP)
# number of pts to retain in Europe
(othDen/eurDen) * nrow(eurOcc)

# thin occurrence records inside Europe
set.seed(123)
eurSub <- eurOcc[sample(nrow(eurOcc), 227), ]
# check density
eurSubSp <- SpatialPoints(eurSub)
eur2PPP <- as.ppp(eurSubSp)
intensity(othPPP)
intensity(eur2PPP)

# combine pts
sp <- rbind(othOcc, eurSub)

### B. Kernel density estimate and background points <a id='kdeBg'></a>

Generate kernel density estimate and sample background points

In [ ]:
# load packages
library(raster)
library(spatialEco)

occ.pts <- read.csv('Shorebird_sampling_proc.csv') # x y or lon lat points of occurrence
occ.pts <- occ.pts[, c('gbifID', 'decimalLongitude', 'decimalLatitude')]
occ.pts$taxa <- "Scolopacidae"
coordinates(occ.pts) <- ~decimalLongitude + decimalLatitude
crs(occ.pts) <- CRS('+init=EPSG:4326')

sp.list <- c("americanus", "arquata", "hudsonicus", "phaeopus")
sp <- sp.list[1]
for(sp in sp.list) {
  #### running bias file ####
  setwd(main)
  setwd('data')
  setwd(sp); sp.dir <- getwd()
  sp.occ <- read.csv(list.files(pattern = 'occurrence'))
  sp.pts <- sp.occ
  coordinates(sp.pts) <- ~longitude + latitude
  
  setwd('current-v1.4-r2.5')
  ref_lay <- raster('bio1.asc') # reference raster layer
  crs(ref_lay) <- CRS('+init=EPSG:4326')
  # WGS 84 lat/lon (EPSG 4326)
  
  bias.pts <- raster::crop(occ.pts, ref_lay)
  if(nrow(bias.pts) > 20000) { # if file size too big, break down thinning
    j.n <- min(floor(nrow(bias.pts) / 10000), 10)
    set.seed(1734)
    bias.list <- bias.pts[sample(1:nrow(bias.pts), 10000 * j.n), ]
    for (j in 1:j.n) {
      bias.sub <- bias.list[(1:10000) + (j-1)*10000, ]
      st <- Sys.time()
      thin.sub <- spThin::thin(cbind(bias.sub, bias.sub@coords), long.col = 'decimalLongitude', lat.col = 'decimalLatitude', spec.col = 'taxa',
                               thin.par = 50, reps = 1, max.files = 1, out.base = sp, locs.thinned.list.return = T,
                               out.dir = op, write.files = F, write.log.file = F, verbose = F)
      bias.sub <- bias.sub[as.integer(rownames(thin.sub[[1]])),]
      if(j == 1){bias.pts <- bias.sub} else {bias.pts <- rbind(bias.pts, bias.sub)}
      et <- Sys.time() - st
      print(paste0(sp, ' time thin ', j, ' of ', round(et, 2), attr(et, "units")))
    }
  }
  ## thinning sampling bias to match occurrence thinning
  bias.thin <- spThin::thin(cbind(bias.pts, bias.pts@coords), long.col = 'decimalLongitude', lat.col = 'decimalLatitude', spec.col = 'taxa',
                            thin.par = 50, reps = 1, max.files = 1, out.base = sp, locs.thinned.list.return = T,
                            out.dir = op, write.files = F, write.log.file = F, verbose = F)
  bias.pts <- bias.pts[as.integer(rownames(bias.thin[[1]])),]
  
  ## generating kernel density estimate (can be single sp) ##
  kdm <- spatialEco::sp.kde(x = bias.pts, # input occurrence pts 
                            bw = 5, # kernel distance
                            newdata = ref_lay, # reference raster
                            standardize = T) # standardize 0 to 1
  ## set crs and crop raster
  kdm <- raster::resample(kdm, ref_lay)
  kdm <- raster::mask(raster::crop(kdm, ref_lay), ref_lay)
  names(kdm) <- "samp_bias"
  setwd(sp.dir)
  raster::writeRaster(kdm, filename = "bias_sampling_bw5.tif")
  
  #### selecting background points ####
  br.buff <- raster(list.files(pattern = 'Br-buffer500km'))
  kdm <- raster::mask(kdm, br.buff) # limitting bg to buffered area only
  
  min.buff <- raster::buffer(sp.pts, 10000) # min distance from occ
  # plot(min.buff)
  raster::crs(min.buff) <- CRS('+init=EPSG:4326')
  min.buff <- raster::mask(kdm, min.buff)
  min.buff[!is.na(min.buff)] <- 0 # raster package format edits
  min.buff[is.na(min.buff)] <- 1 # raster package format edits
  min.buff[min.buff == 0] <- NA # raster package format edits
  
  bg.area <- raster::mask(kdm, min.buff) # mask out area < min distance from occ
  bg.area <- as.data.frame(bg.area, xy = T, cells = F, na.rm = T) # convert to df for sampling
  bg.pts <- bg.area[sample(seq_len(nrow(bg.area)), # row identifier
                           10000, # number of bg points
                           replace = F,
                           prob = bg.area$samp_bias), ] # sampling probability of each row
  write.csv(bg.pts, file = 'bg_pts.csv', row.names = F) # tune results
}


### C. Bioclimatic variables <a id='biovar'></a>

Obtain bioclimatic variables

1. Present day (1960-1990; 30 seconds, bio): https://biogeo.ucdavis.edu/data/climate/worldclim/1_4/grid/cur/bio1-9_30s_bil.zip, https://biogeo.ucdavis.edu/data/climate/worldclim/1_4/grid/cur/bio10-19_30s_bil.zip
2. mid-Holocene (CCSM4, 30 seconds, bi): https://biogeo.ucdavis.edu/data/climate/cmip5/mid/ccmidbi_30s.zip
3. Last Glacial Maximum (CCSM4, 2.5 minutes, bi): https://biogeo.ucdavis.edu/data/climate/cmip5/lgm/cclgmbi_2-5m.zip

Prepare bioclimatic variables

- make sure all rasters have the same nodata value and units
- clip rasters by polygon (SAGA toolbox)
- convert .sdat raster to .adc (Raster > Conversion > Translate)

## 2. Maxent modelling <a id='maxent'></a>

Perform Maxent analyses, model selection, and prediction

In [ ]:
# load packages
library(ENMeval)
library(terra)
library(usdm)
library(ecospat)

main <- "path/workingDirectory"
setwd(main)
setwd('data')
sp.list <- c("americanus", "arquata", "hudsonicus", "phaeopus")
sp <- sp.list[2]
for(sp in sp.list) {
  #### data preparation ####
  setwd(main)
  setwd('data')
  setwd(sp); sp.dir <- getwd()
  ## Occurrences and Background ##
  sp.occ <- read.csv(list.files(pattern = 'occurrence'))
  colnames(sp.occ)[2:3] <- c('x', 'y')
  sp.occ$PA <- 1
  bg.pts <- read.csv('bg_pts.csv')
  bg.pts$PA <- 0
  ip.df <- rbind(sp.occ[,c('x', 'y', 'PA')], bg.pts[,c('x', 'y', 'PA')])
  rm(sp.occ, bg.pts)
  br.buff <- rast(list.files(pattern = 'Br-buffer500km'))
  
  ## Bioclimatic variables ##
  setwd("current-v1.4-r2.5")
  bio.var <- rast(paste0('bio', 1:19, '.asc'))  
  ip.df[, paste0("bio", 1:19)] <- terra::extract(bio.var, ip.df[, c('x', 'y')])[,-1]
  ip.df <- ip.df[!is.na(ip.df$bio1),]
  rm(bio.var)
  
  ## varbiable selection ##
  bio.vif <- usdm::vifstep(ip.df[, paste0("bio", 1:19)], th = 3)
  bio.sel <- bio.vif@results$Variables
  
  ## data partitioning ##
  cv.bins <- get.block(ip.df[ip.df$PA == 1, c('x','y')], ip.df[ip.df$PA == 0, c('x','y')])
  setwd(sp.dir)
  save(cv.bins, file = paste0(sp, "_CVbins"))
  
  
  #### MaxEnt Modelling ####
  setwd(sp.dir)
  betas <- c(0.5, 1, 2, 3, 4)
  fcs <- c("L", "LQ", "LQH", "LQHP")
  
  # start modeling!
    # we use the "try" notation so if a species fails to fit, the loop will continue
  tune_maxent <- NULL # tuned maxent model
  ptm <- proc.time() # record mod time
  # train.df has multiple columns
  # PA = presence (1) or background (0)
  # 'x', 'y' are included as ENMevaluate automatically takes the first two columns as coordinate information even though using dataframe mean that coordinates are ignored
  # varb.list = my list of relevant variables (e.g., bio01, bio02, land_use)
  # Note, categoricals are used here, but user should delineate if there are any
  # user.grp = bins for partitioning the data (cross-validation etc)
  # partitions = 'user' as partitions were set in user.grp
  if(inherits(try(
    tune_maxent <- ENMeval::ENMevaluate(occs = ip.df[ip.df$PA == 1, c('x', 'y', bio.sel)],
                                        bg = ip.df[ip.df$PA == 0, c('x', 'y', bio.sel)],
                                        # categoricals = 'land_use',
                                        user.grp = cv.bins, partitions = 'user',
                                        tune.args = list(fc = fcs, rm = betas),
                                        algorithm = 'maxent.jar', overlap = F, quiet = T,
                                        doClamp = F, parallel = T, numCores = 10)
  ), "try-error")){
    print(paste("MaxEnt Tune: Error for species ", sp))
  }
  tt_maxent <- proc.time() - ptm
  print(tt_maxent)
  
  if(!is.null(tune_maxent)) {
    res_maxent <- tune_maxent@results # extract results table
    write.csv(res_maxent, file = paste0(sp, '_tunemaxent.csv'), row.names = F) # tune results
    
    ## selecting model ##
    mod_maxent <- res_maxent[which(res_maxent$ncoef > 1), ] # remove overly simplistic or non-converging mods
    mod_maxent <- mod_maxent[!is.na(mod_maxent$AICc), ] # remove overly complex mods
    mod_maxent <- mod_maxent[which(mod_maxent$or.mtp.avg <= 0.2), ] # remove underpredicting models
    mod_maxent$cbi.val.CoV <- mod_maxent$cbi.val.sd / mod_maxent$cbi.val.avg
    ## select based on highest AUC score
    mod_maxent <- mod_maxent[which(mod_maxent$auc.val.avg == max(mod_maxent$auc.val.avg, na.rm = T)),] # INCASE > 1, max test AUC
    mod_maxent <- mod_maxent[which(mod_maxent$auc.train == max(mod_maxent$auc.train, na.rm = T)),] # INCASE > 1, max test AUC
    mod_maxent <- tune_maxent@models[[as.numeric(rownames(mod_maxent))[1]]] # extract the selected model from model list
    rm(res_maxent, tune_maxent)
  } else {mod_maxent <- NULL}
  save(mod_maxent, file = paste0(sp, '_selmod'))
  
  #### predicting models ####
  # select different times #
  maxSSS <- mod_maxent@results["Maximum.training.sensitivity.plus.specificity.Cloglog.threshold",]
  for (t.period in 1:3) {
    bio.period <- c('current-v1.4-r2.5', 'LGM-v1.4-r2.5', 'midHolocene-v1.4-r2.5')[t.period]
    nm.period <- c('current', 'LGM', 'MH')[t.period]
    setwd(sp.dir)
    setwd(bio.period)
    bio.var <- rast(paste0(bio.sel, '.asc'))
    bio.var <- terra::mask(bio.var, br.buff)
    setwd(sp.dir)
    if(!file.exists('predictions')) {dir.create('predictions')}
    setwd('predictions')
    pred <- terra::predict(bio.var, mod_maxent, type = "cloglog", na.rm = T) # >= maxSSS.th # if you want binary
    writeRaster(pred, filename = paste0(sp, '_', nm.period, '_cont.tif'), overwrite = T)
    writeRaster(pred >= maxSSS, filename = paste0(sp, '_', nm.period, '_bina.tif'), overwrite = T)
    rm(bio.var)
  }
}

## 3. Maxent results <a id='results'></a>

Calculate suitable breeding area from '_bina.tif'

In [ ]:
# load packages
library(raster)
library(rgdal)
library(maps)
library(ENMTools)
library(rgeos)
library(sp)

# set colour palette and load map
pal <- colorRampPalette(c("#e6e6e6", "#ad8b62", "#f5f4cf", "#c2e075", "#84a338", "#4f6121"))
worldmap <- readOGR(dsn = "path/map", layer = "ne_110m_land")
add.map <- function() {
  plot(worldmap, add=TRUE)
}

# load binary raster
rasterB <- raster("species_time_bina.tif")
# add map outline
plot(rasterB, col = pal(100), addfun=add.map)

## calculate suitable area
# remove cells with suitability score <1 (only 2 values possible as raster has been made binary)
# need to change variable name
raster3<-rasterB[!(rasterB$speices_time_bina < 0.1), ]
length(raster3)
# get sizes of all cells in raster distribution raster
cell_size<-area(rasterB, na.rm=TRUE, weights=FALSE)
# delete NAs from all raster cells
cell_size1<-cell_size[!is.na(cell_size)]
# compute area [km2] of all cells in raster
raster_area_present<-length(raster3)*median(cell_size1)
raster_area_present